In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import shapefile as shp
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

import sys
sys.path.append('../scripts')
from preprocess_script import count_outliers

In [ ]:
# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("Dataset Joining")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.driver.memory", "9g") 
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)
spark.sparkContext.setLogLevel("OFF")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/09/11 07:41:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Dataset downloaded from https://data.gov.au/dataset/ds-dga-2c79581f-600e-4560-80a8-98adb1922dfc/details?q=correspondence%20asgs, data dictionary also in the same.

In [ ]:
correspondence_df = spark.read.csv('../data/tables/correspondence/CG_POSTCODE_2021_SA2_2021.csv', header=True)
correspondence_df.show()

In [ ]:
preferred_SA2 = correspondence_df.groupBy('POSTCODE').agg(F.max('RATIO_FROM_TO'))
preferred_SA2.show()

In [ ]:
preferred_SA2 = preferred_SA2.withColumnRenamed("max(RATIO_FROM_TO)", "RATIO_FROM_TO")

In [ ]:
preferred_SA2.show()
preferred_SA2.count()
preferred_SA2.dropna()
preferred_SA2.count()

In [ ]:
preferred_SA2 = preferred_SA2.join(correspondence_df, on=["POSTCODE", "RATIO_FROM_TO"], how = "left")

In [ ]:
preferred_SA2.show()

In [ ]:
preferred_SA2.select('POSTCODE', 'SA2_CODE_2021')

In [ ]:
# Removing Null values
preferred_SA2 = preferred_SA2.na.drop(how = 'any')

In [ ]:
# Information on consumer
consumer_user_detail = spark.read.parquet("../data/tables/part_1/consumer_user_details.parquet")
consumer = spark.read.option("delimiter" , "|").csv("../data/tables/part_1/tbl_consumer.csv", header = "True")

# Information on consumer's fraud probability
consumer_fraud_prob = spark.read.csv("../data/tables/part_1/consumer_fraud_probability.csv", header = "True")

In [ ]:
consumer_user_detail.show()

In [ ]:
consumer.show()

In [ ]:
consumer_fraud_prob.show()

In [ ]:
consumer = consumer.join(consumer_user_detail, on = "consumer_id", how="left")

In [ ]:
consumer = consumer.join(consumer_fraud_prob, on="user_id", how="left")

In [ ]:
consumer.show()

In [ ]:
consumer.schema

In [ ]:
from pyspark.sql.types import DoubleType
consumer = consumer.withColumn('fraud_probability', F.col('fraud_probability').cast(DoubleType()))

In [ ]:
# Filling 0 for fraud probability null.
consumer = consumer.fillna(0, subset=['fraud_probability'])

In [ ]:
max_fraud = consumer.groupby('user_id').agg(F.max('fraud_probability'))

In [ ]:
max_fraud.show()

+-------+----------------------+
|user_id|max(fraud_probability)|
+-------+----------------------+
|     26|    10.604536233608997|
|     29|    15.492409971599415|
|    474|     24.74251000337883|
|    964|    10.000639641611324|
|   1677|     10.69910130025277|
|   1697|                   0.0|
|   1806|                   0.0|
|   1950|    16.924789438758555|
|   2040|    19.150047883546396|
|   2214|      9.06669796313919|
|   2250|      25.5950368734215|
|   2453|                   0.0|
|   2509|     17.64221987321122|
|   2529|     16.84817579923174|
|   2927|    14.570508610444977|
|   3091|    15.428265394555776|
|   3506|    30.199921043599648|
|   3764|      9.72214044955335|
|   4590|     18.88641597368393|
|   4823|                   0.0|
+-------+----------------------+
only showing top 20 rows

+-------+----------------------+
|user_id|max(fraud_probability)|
+-------+----------------------+
|     26|    10.604536233608997|
|     29|    15.492409971599415|
|    474|     24.

In [ ]:
consumer = consumer.join(max_fraud, on='user_id', how='left')
consumer.show()

+-------+-----------+-----------------+--------------------+-----+--------+-----------+--------------+------------------+----------------------+
|user_id|consumer_id|             name|             address|state|postcode|     gender|order_datetime| fraud_probability|max(fraud_probability)|
+-------+-----------+-----------------+--------------------+-----+--------+-----------+--------------+------------------+----------------------+
|      6|     407340|    Karen Chapman|2706 Stewart Oval...|  NSW|    2033|     Female|    2021-12-12|10.459280127078758|    10.459280127078758|
|      7|     511685|     Andrea Jones|   122 Brandon Cliff|  QLD|    4606|     Female|          NULL|               0.0|                   0.0|
|      9|     650435|  Stephanie Reyes|5813 Denise Land ...|  NSW|    2482|     Female|    2021-11-17| 8.531261989227714|     10.58055311139687|
|      9|     650435|  Stephanie Reyes|5813 Denise Land ...|  NSW|    2482|     Female|    2021-12-13| 10.58055311139687|     10.5

In [ ]:
consumer = consumer.select('user_id', 'consumer_id', 'postcode', 'state', 'gender', 'max(fraud_probability)')
consumer.show()

+-------+-----------+--------+-----+-----------+----------------------+
|user_id|consumer_id|postcode|state|     gender|max(fraud_probability)|
+-------+-----------+--------+-----+-----------+----------------------+
|      6|     407340|    2033|  NSW|     Female|    10.459280127078758|
|      7|     511685|    4606|  QLD|     Female|                   0.0|
|      9|     650435|    2482|  NSW|     Female|     10.58055311139687|
|      9|     650435|    2482|  NSW|     Female|     10.58055311139687|
| 142344|     389526|    1143|  NSW|       Male|                   0.0|
| 142348|     553507|    4723|  QLD|Undisclosed|                   0.0|
| 142352|    1220814|    6479|   WA|     Female|                   0.0|
| 142354|     338380|    3138|  VIC|       Male|                   0.0|
| 142357|     746099|    3226|  VIC|     Female|                   0.0|
| 142359|     895156|    3173|  VIC|     Female|                   0.0|
| 284819|     562577|    2175|  NSW|       Male|                

In [ ]:
consumer = consumer.sort('user_id')

In [ ]:
consumer.show()

+-------+-----------+--------+-----+-----------+----------------------+
|user_id|consumer_id|postcode|state|     gender|max(fraud_probability)|
+-------+-----------+--------+-----+-----------+----------------------+
|      1|    1195503|    6935|   WA|     Female|     9.805431136520959|
|      2|     179208|    2782|  NSW|     Female|    10.069850934775245|
|      2|     179208|    2782|  NSW|     Female|    10.069850934775245|
|      3|    1194530|     862|   NT|     Female|     8.300636455314633|
|      4|     154128|    2780|  NSW|     Female|     9.633302411090419|
|      5|     712975|    6355|   WA|     Female|    27.496186536467164|
|      5|     712975|    6355|   WA|     Female|    27.496186536467164|
|      5|     712975|    6355|   WA|     Female|    27.496186536467164|
|      6|     407340|    2033|  NSW|     Female|    10.459280127078758|
|      7|     511685|    4606|  QLD|     Female|                   0.0|
|      8|     448088|    6056|   WA|       Male|                

In [ ]:
consumer.write.mode('overwrite').parquet('../data/curated/customer/')